In [32]:
import http.client
import json
import urllib.parse
import re
import pandas as pd
from shapely.geometry import Point, Polygon, MultiPolygon
import matplotlib.pyplot as plt
import numpy as np
import requests
import time


In [33]:
# Function to parse the curl bash command into usable components
def clean_curl_request(curl_command):
    # Extract the URL
    url_pattern = r"curl '(.*?)'"
    match = re.search(url_pattern, curl_command)
    if match:
        url = match.group(1)
    else:
        raise ValueError("URL not found in the curl command")

    # Extract headers
    headers = {}
    header_pattern = r"-H '([^:]+): (.*?)'"
    for header_match in re.finditer(header_pattern, curl_command):
        header_name = header_match.group(1)
        header_value = header_match.group(2)
        headers[header_name] = header_value

    # Extract data
    data_pattern = r"--data-raw '(.*?)'"
    match = re.search(data_pattern, curl_command)
    data = {}
    if match:
        data_string = match.group(1)
        data_items = data_string.split("&")
        for item in data_items:
            key, value = item.split("=")
            data[key] = urllib.parse.unquote(value)

    # Parse URL for host and path
    parsed_url = urllib.parse.urlparse(url)
    host = parsed_url.netloc
    path = parsed_url.path + ("?" + parsed_url.query if parsed_url.query else "")

    return host, path, data, headers


In [34]:
def send_request(session, url, data, headers, lat, lng):
    # Update latitude and longitude in the data dictionary
    data["lat"] = str(lat)
    data["lng"] = str(lng)
    
    # Measure request time
    start_time = time.time()
    
    response = session.post(url, data=data, headers=headers)
    elapsed_time = time.time() - start_time  # Calculate elapsed time
    
    # Print the response time
    print(f"Response received in {elapsed_time:.2f} seconds for lat: {lat}, lng: {lng}")
    
    # Check for a successful response and return the result
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None


In [ ]:
def read_file():
    # Poligonun icindeki noktaları buraya ekleyecegiz.
    with open('istanbul-admin-level-6.geojson', 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    return data


In [ ]:
def get_properties(data,ilce):
    for i in data["features"]:
        print(i["properties"].get("name", "point"))
        # Polygonları alıyoruz sadece
        if i["geometry"].get("type", "unnamed") != "Point" and i["properties"].get("name", "point") == ilce:
            name = i["properties"].get("name", "point")
            geometry_type = i["geometry"].get("type", "point")
            coordinates = i["geometry"].get("coordinates", "point")
            # Polygon olanları direkt listeye atıyoruz.
            if geometry_type == "Polygon":
                polygons = [Polygon(coordinates[0])]
            # Multi-poly olanları listeler icinde listeye atıyoruz.
            elif geometry_type == "MultiPolygon":
                polygons = [Polygon(polygon[0]) for polygon in coordinates]
            else:
                raise ValueError(f"Unsupported geometry type: {geometry_type}")
    return name,geometry_type,polygons

In [ ]:
def extract_exterior_coordinates_create_lines(aralik=100,polygons):
        for polygon in polygons:
            # Extract exterior coordinates for plotting
            lat_spacing = aralik / 111320  # Approximation: 1 degree latitude ~ 111.32 km
            min_lat = min([min(polygon.exterior.coords.xy[1]) for polygon in polygons])
            max_lat = max([max(polygon.exterior.coords.xy[1]) for polygon in polygons])
            lat_lines = np.arange(min_lat, max_lat, lat_spacing)

            for lat in lat_lines:
                min_lon = min([min(polygon.exterior.coords.xy[0]) for polygon in polygons])
                max_lon = max([max(polygon.exterior.coords.xy[0]) for polygon in polygons])
                lon_spacing = aralik / (111320 * np.cos(np.radians(lat)))  # Adjust for longitude scaling
                lon_points = np.arange(min_lon, max_lon, lon_spacing)

                # Nokta poligonun icinde mi kontrol ediyoruz icindeyse kırmızı degilse yesil 
                for lon in lon_points:
                    point = Point(lon, lat)
                    is_inside = any(polygon.contains(point) for polygon in polygons)
                    if is_inside:
                        red_points.append((lon, lat))
    return lon_points,lat_lines

In [ ]:
def is_dot_inside_the_polygon(point_list):
    
    for lon in lon_points:
     point = Point(lon, lat)
     is_inside = any(polygon.contains(point) for polygon in polygons)
     if is_inside:
         point_list.append((lon, lat))
    
    return red_points

In [4]:
ilce = 'Adalar'
aralik = 100
# Listeyi loop'un icinde yaratıyoruz ki ilce ilce alabilelim.
red_points = []
# # Poligonun icindeki noktaları buraya ekleyecegiz.
# with open('istanbul-admin-level-6.geojson', 'r', encoding='utf-8') as file:
#     data = json.load(file)

#for i in data["features"]:
#    print(i["properties"].get("name", "point"))
#    # Polygonları alıyoruz sadece
#    if i["geometry"].get("type", "unnamed") != "Point" and i["properties"].get("name", "point") =! "Gebze":
#        name = i["properties"].get("name", "point")
#        geometry_type = i["geometry"].get("type", "point")
#        coordinates = i["geometry"].get("coordinates", "point")
#        # Polygon olanları direkt listeye atıyoruz.
#        if geometry_type == "Polygon":
#            polygons = [Polygon(coordinates[0])]
#        # Multi-poly olanları listeler icinde listeye atıyoruz.
#        elif geometry_type == "MultiPolygon":
#            polygons = [Polygon(polygon[0]) for polygon in coordinates]
#        else:
#            raise ValueError(f"Unsupported geometry type: {geometry_type}")

#        for polygon in polygons:
#            # Extract exterior coordinates for plotting
#            lat_spacing = aralik / 111320  # Approximation: 1 degree latitude ~ 111.32 km
#            min_lat = min([min(polygon.exterior.coords.xy[1]) for polygon in polygons])
#            max_lat = max([max(polygon.exterior.coords.xy[1]) for polygon in polygons])
#            lat_lines = np.arange(min_lat, max_lat, lat_spacing)
#
#            for lat in lat_lines:
#                min_lon = min([min(polygon.exterior.coords.xy[0]) for polygon in polygons])
#                max_lon = max([max(polygon.exterior.coords.xy[0]) for polygon in polygons])
#                lon_spacing = aralik / (111320 * np.cos(np.radians(lat)))  # Adjust for longitude scaling
#                lon_points = np.arange(min_lon, max_lon, lon_spacing)
#
#                # Nokta poligonun icinde mi kontrol ediyoruz icindeyse kırmızı degilse yesil 
#                for lon in lon_points:
#                    point = Point(lon, lat)
#                    is_inside = any(polygon.contains(point) for polygon in polygons)
#                    if is_inside:
#                        red_points.append((lon, lat))

        # DataFrame olustur ve verileri ekle
        columns = ["tesis_adi", "il_adi", "sokak_adi", "ilce_adi", "mahalle_adi", "lng", "lat", "id"]
        df = pd.DataFrame(columns=columns)
        for lng, lat in red_points:
            # Send API request
            result = send_request("host", "path", "data", "headers", lat, lng)
            # If the request is successful, process the results
            if isinstance(result, dict) and "features" in result:
                for feature in result["features"]:
                    # Extract relevant properties
                    properties = feature["properties"]
                    row = {
                        "tesis_adi": properties.get("tesis_adi", ""),
                        "il_adi": properties.get("il_adi", ""),
                        "sokak_adi": properties.get("sokak_adi", ""),
                        "ilce_adi": properties.get("ilce_adi", ""),
                        "mahalle_adi": properties.get("mahalle_adi", ""),
                        "lng": properties.get("lng", ""),
                        "lat": properties.get("lat", ""),
                        "id": properties.get("id", "")
                    }
                    # Append the row to the DataFrame
                    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

        # Drop duplicates after each iteration
        df.drop_duplicates(subset=["id"], inplace=True)

        # Save the final DataFrame to a CSV file
        df.to_csv(f"{name}_results.csv", index=False)


Adalar


TypeError: 'str' object does not support item assignment

In [ ]:
curl_command = """
curl 'https://www.turkiye.gov.tr/afet-ve-acil-durum-yonetimi-acil-toplanma-alani-sorgulama?harita=goster&submit' \
  -H 'Accept: application/json, text/javascript, */*; q=0.01' \
  -H 'Accept-Language: tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7' \
  -H 'Connection: keep-alive' \
  -H 'Content-Type: application/x-www-form-urlencoded; charset=UTF-8' \
  -H 'Cookie: ridbb=WyI1ZWZjYWJjN2E5YzNiY2M4ZDY4OWExMGIwN2IwMTcyMmNjZDc5NjUzZDBlOCJd; _uid=1721632846-0ba03683-f088-45be-ab36-f8dd67f8b32d; w3p=1943251136.20480.0000; language=tr_TR.UTF-8; uyari=b8e23373d48d32ab19013491277695691c051a6639e16343c50db3d9f71adf64; TURKIYESESSIONID=l4c0eutds26b2qr2ghqjf9mbt4; TS015d3f68=015c1cbb6d6bcb6358b3fc4d2387f6b25fa2d1245dfddce1e7b22bd8348a14a1e565feedf84c8944203eed55b58f837e9f56aacdc0; _lastptts=1735847586' \
  -H 'Origin: https://www.turkiye.gov.tr' \
  -H 'Referer: https://www.turkiye.gov.tr/afet-ve-acil-durum-yonetimi-acil-toplanma-alani-sorgulama?harita=goster' \
  -H 'Sec-Fetch-Dest: empty' \
  -H 'Sec-Fetch-Mode: cors' \
  -H 'Sec-Fetch-Site: same-origin' \
  -H 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 OPR/115.0.0.0' \
  -H 'X-Requested-With: XMLHttpRequest' \
  -H 'sec-ch-ua: "Chromium";v="130", "Opera";v="115", "Not?A_Brand";v="99"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Windows"' \
  --data-raw 'pn=%2Fafet-ve-acil-durum-yonetimi-acil-toplanma-alani-sorgulama&ajax=1&token=%7B6D7A6F-2ED4EB-F96A69-A23803-F7E63C-51CA4D-9AB3C3-BCD5F2%7D&islem=getAlanlarForNokta&lat=40.969583296311086&lng=29.095801512231148'
"""


# Parse the curl command
host, path, data, headers = clean_curl_request(curl_command)

In [ ]:


# Read coordinates from file and process them
with open('coordinates.txt', 'r') as file:
    for line in file:
        try:
            # Extract longitude and latitude
            lng, lat = map(float, line.strip().split(', '))

            # Send API request
            result = send_request(host, path, data, headers, lat, lng)

            # If the request is successful, process the results
            if isinstance(result, dict) and "features" in result:
                for feature in result["features"]:
                    # Extract relevant properties
                    properties = feature["properties"]
                    row = {
                        "tesis_adi": properties.get("tesis_adi", ""),
                        "il_adi": properties.get("il_adi", ""),
                        "sokak_adi": properties.get("sokak_adi", ""),
                        "ilce_adi": properties.get("ilce_adi", ""),
                        "mahalle_adi": properties.get("mahalle_adi", ""),
                        "lng": properties.get("lng", ""),
                        "lat": properties.get("lat", ""),
                        "id": properties.get("id", "")
                    }
                    # Append the row to the DataFrame
                    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

            # Drop duplicates after each iteration
            df.drop_duplicates(subset=["id"], inplace=True)

        except Exception as e:
            print(f"Error processing line '{line.strip()}': {e}")

# Save the final DataFrame to a CSV file
df.to_csv("api_results.csv", index=False)


In [35]:
import json
import numpy as np
import pandas as pd
from shapely.geometry import Polygon, Point

def load_geojson(filepath, ilce):
    with open(filepath, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return [feature for feature in data["features"] if feature["properties"].get("name") == ilce]

def extract_polygons(features):
    polygons = []
    for feature in features:
        geometry_type = feature["geometry"].get("type")
        coordinates = feature["geometry"].get("coordinates")
        if geometry_type == "Polygon":
            polygons.extend([Polygon(coordinates[0])])
        elif geometry_type == "MultiPolygon":
            polygons.extend([Polygon(polygon[0]) for polygon in coordinates])
        else:
            raise ValueError(f"Unsupported geometry type: {geometry_type}")
    return polygons

def generate_grid(polygons, aralik):
    red_points = []
    lat_spacing = aralik / 111320
    for polygon in polygons:
        min_lat = min(polygon.exterior.coords.xy[1])
        max_lat = max(polygon.exterior.coords.xy[1])
        lat_lines = np.arange(min_lat, max_lat, lat_spacing)
        for lat in lat_lines:
            min_lon = min(polygon.exterior.coords.xy[0])
            max_lon = max(polygon.exterior.coords.xy[0])
            lon_spacing = aralik / (111320 * np.cos(np.radians(lat)))
            lon_points = np.arange(min_lon, max_lon, lon_spacing)
            for lon in lon_points:
                point = Point(lon, lat)
                if any(polygon.contains(point) for polygon in polygons):
                    red_points.append((lon, lat))
    return red_points,polygons

def process_api_requests(red_points, send_request_func, polygons):
    columns = ["tesis_adi", "il_adi", "sokak_adi", "ilce_adi", "mahalle_adi", "lng", "lat", "id", "polygon_bounds"]
    df = pd.DataFrame(columns=columns)
    for lng, lat in red_points:
        result = send_request_func(lat, lng)
        if isinstance(result, dict) and "features" in result:
            for feature in result["features"]:
                properties = feature["properties"]
                row = {
                    "tesis_adi": properties.get("tesis_adi", ""),
                    "il_adi": properties.get("il_adi", ""),
                    "sokak_adi": properties.get("sokak_adi", ""),
                    "ilce_adi": properties.get("ilce_adi", ""),
                    "mahalle_adi": properties.get("mahalle_adi", ""),
                    "lng": lng,
                    "lat": lat,
                    "id": properties.get("id", ""),
                    "polygon_bounds": polygons  # Now passed explicitly
                }
                df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    return df.drop_duplicates(subset=["id"])

def save_dataframe(df, filename):
    """
    Saves the given DataFrame to a CSV file.

    Args:
        df (pd.DataFrame): The DataFrame to save.
        filename (str): The name of the CSV file.
    """
    try:
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"Data saved successfully to {filename}")
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")



def process_district(ilce, aralik, geojson_filepath, curl_command):
    # Parse the curl command
    host, path, data, headers = clean_curl_request(curl_command)
    url = f"https://{host}{path}"
    
    # Load GeoJSON and process data
    features = load_geojson(geojson_filepath, ilce)
    polygons = extract_polygons(features)
    red_points, polygons = generate_grid(polygons, aralik)  # Unpack both
    
    # Create a session and update headers
    session = requests.Session()
    session.headers.update(headers)
    
    # Define the function for API requests
    send_request_func = lambda lat, lng: send_request(session, url, data, headers, lat, lng)
    
    # Process API requests
    df = process_api_requests(red_points, send_request_func, polygons)  # Pass polygons explicitly
    print(df.head(10))
    
    # Save results
    save_dataframe(df, f"./results/{ilce}_results.csv")









In [ ]:
# Example usage
curl_command = """
curl 'https://www.turkiye.gov.tr/afet-ve-acil-durum-yonetimi-acil-toplanma-alani-sorgulama?harita=goster&submit' \
  -H 'Accept: application/json, text/javascript, */*; q=0.01' \
  -H 'Accept-Language: tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7' \
  -H 'Connection: keep-alive' \
  -H 'Content-Type: application/x-www-form-urlencoded; charset=UTF-8' \
  -H 'Cookie: ridbb=WyI1ZWZjYWJjN2E5YzNiY2M4ZDY4OWExMGIwN2IwMTcyMmNjZDc5NjUzZDBlOCJd; _uid=1721632846-0ba03683-f088-45be-ab36-f8dd67f8b32d; w3p=1943251136.20480.0000; language=tr_TR.UTF-8; uyari=b8e23373d48d32ab19013491277695691c051a6639e16343c50db3d9f71adf64; TURKIYESESSIONID=vi8iibo85cbau3nd1v2umiu8av; TS015d3f68=015c1cbb6dc22950c6d3d04363ff6c1ac0c32cf262a64bdc128e5b4d0ce98818bc21f7c0229b4d3ddaf361b080f25b9b347e4267c3; _lastptts=1736153030' \
  -H 'Origin: https://www.turkiye.gov.tr' \
  -H 'Referer: https://www.turkiye.gov.tr/afet-ve-acil-durum-yonetimi-acil-toplanma-alani-sorgulama?harita=goster' \
  -H 'Sec-Fetch-Dest: empty' \
  -H 'Sec-Fetch-Mode: cors' \
  -H 'Sec-Fetch-Site: same-origin' \
  -H 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 OPR/115.0.0.0' \
  -H 'X-Requested-With: XMLHttpRequest' \
  -H 'sec-ch-ua: "Chromium";v="130", "Opera";v="115", "Not?A_Brand";v="99"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Windows"' \
  --data-raw 'pn=%2Fafet-ve-acil-durum-yonetimi-acil-toplanma-alani-sorgulama&ajax=1&token=%7B86E2F3-42F42D-DD27CE-71D697-A99AAF-1B5ABC-2F066C-D749B6%7D&islem=getAlanlarForNokta&lat=40.967710436913684&lng=29.09652704986536'
"""
# Replace `send_request` with your actual API call function.
process_district("Silivri", 100, "istanbul-admin-level-6.geojson", curl_command)

Response received in 0.42 seconds for lat: 41.032956611174995, lng: 28.007121688854845


C:\Users\boran\AppData\Local\Temp\ipykernel_27340\3226319774.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)


Response received in 0.23 seconds for lat: 41.032956611174995, lng: 28.00831255848334
Response received in 0.23 seconds for lat: 41.03385492234999, lng: 28.00593129046807
Response received in 0.22 seconds for lat: 41.03385492234999, lng: 28.00712217634628
Response received in 0.21 seconds for lat: 41.03385492234999, lng: 28.008313062224488
Response received in 0.21 seconds for lat: 41.03385492234999, lng: 28.009503948102697
Response received in 0.21 seconds for lat: 41.03475323352498, lng: 28.00712266385988
Response received in 1.55 seconds for lat: 41.03475323352498, lng: 28.008313565988544
Response received in 0.21 seconds for lat: 41.03475323352498, lng: 28.009504468117207
Response received in 0.84 seconds for lat: 41.03475323352498, lng: 28.01069537024587
Response received in 0.20 seconds for lat: 41.03475323352498, lng: 28.011886272374532
Response received in 0.20 seconds for lat: 41.03475323352498, lng: 28.014268076631858
Response received in 0.21 seconds for lat: 41.035651544699